#### Import standard required modules

In [ ]:
import control.matlab as c
import matplotlib.pyplot as plt
import numpy as np
import copy

In [ ]:
#This keeps the notebook clean
import warnings
warnings.filterwarnings('ignore')

#### Import module 'functions'.
This contains functions based on provided matlab code, such as 'create_SS', which is based on cit2s.m and creates a state space system that includes gust effects. This module can be found in the same folder as this notebook.

In [ ]:
import functions

### 1. Stability Analysis
#### 1.a Creating the state space systems
Creating the undamped state space system first, gust included:

In [ ]:
undamped_sys = functions.create_SS()

The damped system is a copy of the undamped system that includes an active feedback loop with the control law $\delta_e = K_{\theta}$. The value of $K_\theta$ should be chosen such that the damping ratio of the phugoid is equal to $0.5$  and can be determined with the root locus plot of the undamped system.

In [ ]:
rlist, klist = c.rlocus(undamped_sys[2,0], kvect=np.linspace(0, -50.0, num=1000),xlim=[-1.4,0],ylim=[-2.5,2.5])

A downside of this web-based notebook is that this root locus plot is not interactive. An offline (interactive and clickable) version shows that the required gain lies around -0.11. This can be shown here as well:

In [ ]:
#Gain values before and after -0.11:
gain1 = klist[2]
gain2 = klist[3]

#Define s variable
s = c.tf([1, 1],[1])-1

#Every gain value has a number of root locations plotted (1 for every root locus section). 
#Get the two roots from the phugoid sections and multiply them with each other to find 
#the characteristic equations:
char_equation1 = (s - rlist[2][5])*(s - rlist[2][6])
char_equation2 = (s - rlist[3][5])*(s - rlist[3][6])

#Characteristic equations are now in transfer function form, retrieve the numerators:
num1, dummy1 = c.tfdata(char_equation1)
num2, dummy2 = c.tfdata(char_equation2)

#Recall that the characteristic equation has standard form s^2 + 2*zeta*wn + wn^2 and
#determine zetas:
zeta1 = num1[0][0][1]/(2*(num1[0][0][2]**0.5))
zeta2 = num2[0][0][1]/(2*(num2[0][0][2]**0.5))

print "For a gain value of "+str(round(gain1,5))+", the damping ratio becomes "+str(round(zeta1.real,5))+"."
print "For a gain value of "+str(round(gain2,5))+", the damping ratio becomes "+str(round(zeta2.real,5))+"."




The proper gain value is therefore somewhere in between.

Finetuning this process with trial and error in Python eventually results in a more exact gain value of -0.117017. The damped system can now be found:

In [ ]:
#First create a copy of the undamped system:
damped_sys = copy.deepcopy(undamped_sys)

#Create a feedback vector with K_theta at the theta position.
K_theta = -0.117017
K = [0, 0, K_theta, 0, 0, 0, 0]

#Add the feedback loop:
damped_sys.A = undamped_sys.A - undamped_sys.B[:,0]*K


Now there are state space systems for the damped and undamped case. Comparing the damping ratios of the phugoid eigenmotions shows that the damped system is properly damped:

In [ ]:
#Round damping ratio to 5 nearest digits and select the damping ratios
#that belong to the phugoid eigenmotions.
zeta_undamped = round(c.damp(undamped_sys,doprint=False)[1][2],5)
zeta_damped = round(c.damp(damped_sys,doprint=False)[1][2],5)

print "Undamped damping ratio: "+str(zeta_undamped)
print "Damped damping ratio: "+str(zeta_damped)

#### 1.b) Analysis of results
A quick comparison of the step responses of both systems can give an indication of the stability of the systems.

In [ ]:
#Create a large enough time scale
t = np.arange(0,100,0.01)

#Retrieve step response to default inputs
[y1, t] = c.step(undamped_sys[2,0], t)
[y2, t] = c.step(damped_sys[2,0], t)
                 
#Plot the figure
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(t, -y1, label="Undamped", color='b')
ax.plot(t, -y2, label="Damped", color='r')
ax.set_xlim([0,100])
plt.xlabel('Time [s]')
plt.ylabel(r"$\theta$ [$\degree$]")
plt.legend(loc='upper right')
plt.title("Step response comparison")
plt.grid()
plt.show()

Both systems appear stable and the system with the feedback loop is much better damped. Pole-zero maps confirm that both systems are stable:

In [ ]:
p, z = c.pzmap(undamped_sys[2,0], title="Undamped Pole Zero Map")

print "Undamped short period eigenvalues:"
print c.damp(undamped_sys,doprint=False)[2][0]
print c.damp(undamped_sys,doprint=False)[2][1]
print
print "Undamped phugoid eigenvalues:"
print c.damp(undamped_sys,doprint=False)[2][2]
print c.damp(undamped_sys,doprint=False)[2][3]

All poles are located in the left-hand pane. The same is true for the damped pole-zero map:

In [ ]:
p, z = c.pzmap(damped_sys[2,0], title="Damped Pole Zero Map")

print "Damped short period eigenvalues:"
print c.damp(damped_sys,doprint=False)[2][0]
print c.damp(damped_sys,doprint=False)[2][1]
print
print "Damped phugoid eigenvalues:"
print c.damp(damped_sys,doprint=False)[2][2]
print c.damp(damped_sys,doprint=False)[2][3]

All the eigenvalues of both the damped and undamped short periods and phugoids have positive real parts, so both systems are longitudinally stable.